### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [14]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [15]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [16]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [17]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [18]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


#### 1. Similaridad de Documentos

In [37]:
def get_similar_docs(idx):
    # Medimos similaridad coseno entre el documento seleccionado y todos los demás
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Obtenemos los índices de los 5 documentos más similares (excluyendo el mismo documento)
    mostsim = np.argsort(cossim)[::-1][1:6]

    # Mostramos info del documento seleccionado
    print(f"\nDocumento original (ID: {idx} / Clase: {newsgroups_train.target_names[y_train[idx]]}):")
    print(f"{newsgroups_train.data[idx][:500]}...")  # Muestra un extracto del documento
    print("\n-----------------------------------\n")

    # Mostramos info de los documentos similares encontrados
    for i, sim_idx in enumerate(mostsim):
        print(f"Documento similar {i+1} (ID: {sim_idx} / Clase: {newsgroups_train.target_names[y_train[sim_idx]]}):")
        print(f"{newsgroups_train.data[sim_idx][:500]}...")  # Muestra un extracto del documento
        print("\n-----------------------------------\n")

In [29]:
# Seleccionamos 5 documentos al azar
random_indices = np.random.choice(X_train.shape[0], size=5, replace=False)
print(random_indices)

[ 4714  2963 10743   552  5219]


In [38]:
get_similar_docs(random_indices[0])


Documento original (ID: 4714 / Clase: rec.sport.hockey):
from Anna Matyas:

Yeah, and also be second in the team in scoring and play about 35
minutes a game and play on the power play and kill penalties and be the
best defenseman in the league. I'd take a whole team of Chelioses if I
could. (That way, when one got a penalty the others could kill it!)...

-----------------------------------

Documento similar 1 (ID: 4541 / Clase: rec.sport.hockey):

I've never seen a game where one player has committed 5
penalties.  Something like this would require more attention by
the referee.


But you're creating a scoring opportunity where there might not
have been one before.  I can see the relationship between free
throws awarded after a certain number of fouls, but it's
obviously easier to score in basketball, there are more
opportunities.  If a basketball team scores 100 points, that's
at least fifty chances made.  The average number of shots tak...

-----------------------------------

Docum

In [39]:
get_similar_docs(random_indices[1])


Documento original (ID: 2963 / Clase: sci.med):
Probably within 50 years, a new type of eugenics will be possible.
Maybe even sooner.  We are now mapping the human genome.  We will
then start to work on manipulation of that genome.  Using genetic
engineering, we will be able to insert whatever genes we want.
No breeding, no "hybrids", etc.  The ethical question is, should
we do this?  Should we make a race of disease-free, long-lived,
Arnold Schwartzenegger-muscled, supermen?  Even if we can.



-- 
-------------------------------------------...

-----------------------------------

Documento similar 1 (ID: 8550 / Clase: sci.med):


So just what was it you wanted to say?



-- 
----------------------------------------------------------------------------
Gordon Banks  N3JXP      | "Skepticism is the chastity of the intellect, and
geb@cadre.dsl.pitt.edu   |  it is shameful to surrender it too soon." ...

-----------------------------------

Documento similar 2 (ID: 8660 / Clase: sci.med

In [40]:
get_similar_docs(random_indices[2])


Documento original (ID: 10743 / Clase: misc.forsale):
Hello,

  I have three software packages for sale:

  1. The Norton pcANYWHERE version 4.5 for dos. This software
     include host and remote software. It can control both
     dos and windows 3.1 (both standard and enhanced modes)
     activities. Registration card is enclosed.

     Price: $40.00

  2. The Norton AntiVirus for windows and dos version 2.1. It
     can update virus information any time. It also support
     networks. Registration card is enclosed.

     Price: $40.00

  3. Delt...

-----------------------------------

Documento similar 1 (ID: 1584 / Clase: misc.forsale):
I'm looking to find some people interested in getting some cd-rom's. Below
is a list with their prices. If you are interested in any of these, send me
some mail and I can guarantee this price. If you are not local their will be
a shipping cost, and cod cost if you prefer it to be shipped that way.
Marcus

Updated prices from last post.

      
Ame

In [41]:
get_similar_docs(random_indices[3])


Documento original (ID: 552 / Clase: comp.graphics):
Toronto Siggraph 

What: ``Chance's Art'': 2D Graphics and Animation on the Indigo.

By:    Ken Evans, Imagicians Artware, Inc. 

When:  Tuesday 20 April 1993 7:00pm-9:00pm 

Where: The McLuhan Centre for Culture and Technology
       University of Toronto
       39A Queen's Park Crescent
       Toronto

Who:   Members and non-members alike 
       (non-members encouraged to become members...)

Abstract:

Imagicians Artware, Inc. is entering into early beta site testing on Silic...

-----------------------------------

Documento similar 1 (ID: 5856 / Clase: sci.crypt):

Thanks for posting this and making it available. This post will be LONG, I will
comment on most of it, and am reluctantly leaving all of the original in place
to provide context.

Please note that an alt. group has been set up for the Clipper stuff.

                                                     ^^^^^^^^^
Hum, AT&T, VLSI and Mykotronx are 'industry'?
Wonder wh

In [42]:
get_similar_docs(random_indices[4])


Documento original (ID: 5219 / Clase: comp.windows.x):

Here I am!!

-------------------------------------------------------------------------
David Smyth				david@jpl-devvax.jpl.nasa.gov
Senior Software Engineer,		(818)306-6463 (temp! do NOT use v-mail)
X and Object Guru.			tempory office: 525/B70
Jet Propulsion Lab, M/S 525-3660 4800 Oak Grove Drive, Pasadena, CA 91109
------------------------------------------------------------------------- 
 What's the earliest possible date you can't prove it won't be done by?
					- Tom DeMarco...

-----------------------------------

Documento similar 1 (ID: 6047 / Clase: comp.windows.x):

Is this an advantage to MS Windows or to Xt?  I used to think it
was a big advantage for Xt, but I am not at all sure anymore...

-------------------------------------------------------------------------
David Smyth				david@jpl-devvax.jpl.nasa.gov
Senior Software Engineer,		(818)306-6463 (temp! do NOT use v-mail)
X and Object Guru.			tempory office: 525/B70


El análisis de similaridad coseno mostró que los documentos más similares suelen pertenecer a la misma categoría, lo que indica que la vectorización captura bien las características distintivas de cada grupo. No obstante, en algunos casos, la similaridad fue alta entre documentos de categorías diferentes, lo que sugiere que la métrica puede verse influenciada por términos comunes sin considerar completamente el contexto. En general, la técnica es efectiva, pero tiene limitaciones en la detección de matices semánticos más profundos.

#### 2. Entrenamiento de Modelos de Naïve Bayes

In [61]:
# Diferentes configuraciones del vectorizador
vectorizers = [
    TfidfVectorizer(ngram_range=(1, 1), stop_words='english'),
    TfidfVectorizer(ngram_range=(1, 2), stop_words='english'),
    TfidfVectorizer(ngram_range=(2, 2), stop_words='english'),
    TfidfVectorizer(ngram_range=(1, 3), stop_words='english')
]

In [62]:
# Diferentes configuraciones del modelo de Naïve Bayes
models = [
    MultinomialNB(alpha=1.0),
    MultinomialNB(alpha=0.5),
    ComplementNB(alpha=1.0),
    ComplementNB(alpha=0.5)
]

In [63]:
best_score = 0
best_model = None
best_vectorizer = None

# Iteramos sobre todas las combinaciones de vectorizadores y modelos
for vectorizer in vectorizers:
    # Vectorizamos los datos de entrenamiento y test
    X_train_vect = vectorizer.fit_transform(newsgroups_train.data)
    X_test_vect = vectorizer.transform(newsgroups_test.data)

    for model in models:
        # Entrenamos el modelo
        model.fit(X_train_vect, newsgroups_train.target)

        # Predecimos sobre el conjunto de test
        y_pred = model.predict(X_test_vect)

        # Calculamos el f1-score macro
        score = f1_score(newsgroups_test.target, y_pred, average='macro')
        print(f"Vectorizer: TfidfVectorizer con ngram_range {vectorizer.ngram_range}, Model: {type(model).__name__} con alpha {model.alpha}, F1-score (macro): {score:.4f}")

        # Guardamos el mejor modelo y configuración
        if score > best_score:
            best_score = score
            best_model = model
            best_vectorizer = vectorizer

print(f"\nMejor F1-score (macro): {best_score:.4f}")
print(f"Mejor Vectorizador: TfidfVectorizer con ngram_range {best_vectorizer.ngram_range}")
print(f"Mejor Modelo: {type(best_model).__name__} con alpha {best_model.alpha}")

Vectorizer: TfidfVectorizer con ngram_range (1, 1), Model: MultinomialNB con alpha 1.0, F1-score (macro): 0.6468
Vectorizer: TfidfVectorizer con ngram_range (1, 1), Model: MultinomialNB con alpha 0.5, F1-score (macro): 0.6584
Vectorizer: TfidfVectorizer con ngram_range (1, 1), Model: ComplementNB con alpha 1.0, F1-score (macro): 0.6936
Vectorizer: TfidfVectorizer con ngram_range (1, 1), Model: ComplementNB con alpha 0.5, F1-score (macro): 0.6978
Vectorizer: TfidfVectorizer con ngram_range (1, 2), Model: MultinomialNB con alpha 1.0, F1-score (macro): 0.6425
Vectorizer: TfidfVectorizer con ngram_range (1, 2), Model: MultinomialNB con alpha 0.5, F1-score (macro): 0.6545
Vectorizer: TfidfVectorizer con ngram_range (1, 2), Model: ComplementNB con alpha 1.0, F1-score (macro): 0.7039
Vectorizer: TfidfVectorizer con ngram_range (1, 2), Model: ComplementNB con alpha 0.5, F1-score (macro): 0.7105
Vectorizer: TfidfVectorizer con ngram_range (2, 2), Model: MultinomialNB con alpha 1.0, F1-score (ma

#### 3. Similaridad entre Palabras

In [95]:
# Transponemos la matriz documento-término para obtener la matriz término-documento
X_train_transposed = X_train.T

# Creamos un diccionario para mapear índices a palabras
idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}

# Seleccionamos 5 palabras para estudiar su similaridad
selected_words = ['football', 'food', 'science', 'religion', 'queen']
selected_word_indices = [tfidfvect.vocabulary_[word] for word in selected_words]

# Calculamos la similaridad entre las palabras seleccionadas y todas las demás palabras
for word_idx in selected_word_indices:
    cossim = cosine_similarity(X_train_transposed[word_idx], X_train_transposed)[0]

    # Obtener los índices de las 5 palabras más similares (excluyendo la misma palabra)
    most_similar = np.argsort(cossim)[::-1][1:6]

    print(f"\nPalabra original: {idx2word[word_idx]}")
    for i, sim_idx in enumerate(most_similar):
        print(f"Palabra similar {i+1}: {idx2word[sim_idx]} (similaridad: {cossim[sim_idx]:.4f})")


Palabra original: football
Palabra similar 1: madden (similaridad: 0.3741)
Palabra similar 2: palying (similaridad: 0.3342)
Palabra similar 3: zabriskie (similaridad: 0.3256)
Palabra similar 4: sportstalk (similaridad: 0.3173)
Palabra similar 5: basketball (similaridad: 0.2896)

Palabra original: food
Palabra similar 1: msg (similaridad: 0.4403)
Palabra similar 2: eat (similaridad: 0.3300)
Palabra similar 3: chinese (similaridad: 0.3057)
Palabra similar 4: flavor (similaridad: 0.3011)
Palabra similar 5: arghhhhhhhhhh (similaridad: 0.2980)

Palabra original: science
Palabra similar 1: behaviorists (similaridad: 0.3948)
Palabra similar 2: cognitivists (similaridad: 0.3948)
Palabra similar 3: scientific (similaridad: 0.3466)
Palabra similar 4: empirical (similaridad: 0.2810)
Palabra similar 5: sects (similaridad: 0.2622)

Palabra original: religion
Palabra similar 1: religious (similaridad: 0.2451)
Palabra similar 2: religions (similaridad: 0.2116)
Palabra similar 3: categorized (similar

El análisis de similaridad coseno entre palabras reveló patrones interesantes:

- football: Palabras similares como madden y basketball están relacionadas con el ámbito deportivo.

- food: Se capturan términos relevantes en el contexto de la comida como eat, chinese y flavor.

- science: Las palabras más similares, como behaviorists y cognitivists, están relacionadas con campos científicos.

- religion: Las palabras similares como religious y crusades están vinculadas temáticamente.

- queen: Los resultados incluyen palabras no intuitivamente relacionadas, en este caso nombres de cantantes, que dependen mucho del contexto en que se utilizan.

En general, la técnica de similaridad coseno proporciona una visión útil de las relaciones semánticas entre palabras, aunque los resultados pueden variar en términos de relevancia directa y contexto, destacando la importancia de considerar el contexto en el análisis.